## <center>데이터 수집 - 전처리 - 모델구현</center>

### 이미지 데이터 크롤링
#### 상어와 고래의 이미지를 수집

In [1]:
#from urllib.parse import quote_plus           # url 구문 분석을 위한 패키지
#from bs4 import BeautifulSoup as bs           # url을 구성요소로 구문 분석하기 위한 패키지

In [2]:
#라이브러리 불러오기
from selenium import webdriver
import time
from urllib.request import (urlopen, urlparse, urlunparse, urlretrieve)

In [3]:
# 구글 이미지 url
# 크롬 드라이브 경로가 너무 복잡하면 에러가나기 떄문에 새로 인스톨 후 가까운 경로로 설정

chrome_path ='C:\chromedriver-win64\chromedriver-win64\chromedriver.exe'
base_url = "https://www.google.co.kr/imghp"

In [4]:

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # 창 없는 모드
# headless 모드의 호환성을 위해 아래 옵션 추가(가끔 막는 웹이 있음)
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
chrome_options.add_argument("lang=ko_KR") # 한국어
chrome_options.add_argument('window-size=1920x1080')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(chrome_path,chrome_options=chrome_options)
driver.get(base_url)
driver.implicitly_wait(3) # element가 로드될 때까지 지정한 시간만큼 대기할 수 있도록 설정
driver.get_screenshot_as_file('google_screen.png')
driver.close()

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_2144\2612864465.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_path,chrome_options=chrome_options)


In [5]:
def selenium_scroll_option():
  SCROLL_PAUSE_SEC = 3
  
  # 스크롤 높이 가져옴
  last_height = driver.execute_script("return document.body.scrollHeight")
  
  while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
  
    if new_height == last_height:
        break
    last_height = new_height


In [6]:
# 검색어로 이미지 크롤링 하기
flatfish = './fish_image_dataset/flatfish/'
korean_rockfish = './fish_image_dataset/korean_rockfish/'
black_sea_bream= './fish_image_dataset/black_sea_bream/'
rock_bream= './fish_image_dataset/rock_bream/'
greenling= './fish_image_dataset/greenling/'

#### xpath를 요소로 정할 때 인덱스를 지정해줘야 했음

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import urllib.request
import os
import pandas as pd

# 키워드 검색하기

a=input("검색할 키워드를 입력 : ")
image_name = input("저장할 이미지 이름 : ")
#b=int(input("몇 개 저장할래? : "))
driver = webdriver.Chrome(chrome_path)
driver.get('http://www.google.co.kr/imghp?hl=ko')
browser = driver.find_element_by_name("q")
browser.send_keys(a)
browser.send_keys(Keys.RETURN)

PAUSE_TIME = 2

# 클래스를 찾고 해당 클래스의 src 리스트를 만들자

selenium_scroll_option() # 스크롤하여 이미지를 많이 확보
#driver.find_elements_by_xpath('//*[@id="islmp"]/div/div/div/div/div[1]/div[2]/div[2]/input')[0].click() # 이미지 더보기 클릭
#selenium_scroll_option()



'''이미지 src요소를 리스트업해서 이미지 url 저장'''

#images = driver.find_elements_by_css_selector(".rg_i.Q4LuWd") #  클래스 네임에서 공백은 .을 찍어줌
#images_url = []
#for i in images: 
#   
#   if i.get_attribute('src')!= None :
#        images_url.append(i.get_attribute('src'))
#   else :
#       images_url.append(i.get_attribute('data-src'))
        
img_elements = driver.find_elements(By.CSS_SELECTOR,".rg_i")
#imgs = []
images_url = []

for idx, img in enumerate(img_elements) :
    print(f"{image_name} : {idx+1}/{len(img_elements)} proceed...")
    try :
        img.click()
        time.sleep(PAUSE_TIME)
        # 이부분에서 에러나면, 직접 개발자 도구 활용해서 XPATH 추출한 뒤에 변경
        img_element = driver.find_element(By.XPATH,'//*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/div[3]/div[1]/a/img[1]')
        # img_element = driver.find_element(By.XPATH,'//*[@id="Sva75c"]/div[2]/div/div[2]/div[2]/div[2]/c-wiz/div/div[2]/div[1]/a/img[1]')
        # //*[@id="Sva75c"]/div[2]/div[2]/div[2]/div[2]/c-wiz/div/div/div/div/div[3]/div[1]/a/img[1]
        #img_src = img_element.get_attribute('src')
        #img_alt = img_element.get_attribute('alt')
        
        if img_element.get_attribute('src')!= None :
            images_url.append(img_element.get_attribute('src'))
        else :
            images_url.append(img_element.get_attribute('data-src'))
        
        #imgs.append({
        #    'alt' : img_alt,
        #    'src' : img_src
        #})
        
    except :
        print(f'err in {idx}')
        pass
#driver.close()



# 겹치는 이미지 url 제거

print("전체 다운로드한 이미지 개수: {}\n동일한 이미지를 제거한 이미지 개수: {}".format(len(images_url), len(pd.DataFrame(images_url)[0].unique())))
images_url=pd.DataFrame(images_url)[0].unique()

#import requests
        
#'''해당하는 파일에 이미지 다운로드'''        
#        
if image_name == 'flatfish' :   
   for t, url in enumerate(images_url, 0):
#      r = requests.get(url, verify=False)
#      with open(flatfish + image_name + '_' + str(t) + '.jpg', "wb") as outfile:
#        outfile.write(r.content)
      urlretrieve(url, flatfish + image_name + '_' + str(t) + '.jpg')
   driver.close()
   
elif image_name == 'korean_rockfish' :
   for t, url in enumerate(images_url, 0):        
      urlretrieve(url, korean_rockfish + image_name + '_' + str(t) + '.jpg')
   driver.close()    

elif image_name == 'black_sea_bream' :
   for t, url in enumerate(images_url, 0):        
      urlretrieve(url, black_sea_bream + image_name + '_' + str(t) + '.jpg')
   driver.close()

elif image_name == 'rock_bream' :
   for t, url in enumerate(images_url, 0):        
      urlretrieve(url, rock_bream + image_name + '_' + str(t) + '.jpg')
   driver.close()    

elif image_name == 'greenling' :
   for t, url in enumerate(images_url, 0):        
      urlretrieve(url, greenling + image_name + '_' + str(t) + '.jpg')
   driver.close()


검색할 키워드를 입력 : 넙치
저장할 이미지 이름 : flatfish
flatfish : 1/400 proceed...
flatfish : 2/400 proceed...
flatfish : 3/400 proceed...
flatfish : 4/400 proceed...
flatfish : 5/400 proceed...
flatfish : 6/400 proceed...
flatfish : 7/400 proceed...
flatfish : 8/400 proceed...
flatfish : 9/400 proceed...
flatfish : 10/400 proceed...
flatfish : 11/400 proceed...
flatfish : 12/400 proceed...
flatfish : 13/400 proceed...
flatfish : 14/400 proceed...
flatfish : 15/400 proceed...
flatfish : 16/400 proceed...
flatfish : 17/400 proceed...
flatfish : 18/400 proceed...
flatfish : 19/400 proceed...
flatfish : 20/400 proceed...
flatfish : 21/400 proceed...
flatfish : 22/400 proceed...
flatfish : 23/400 proceed...
flatfish : 24/400 proceed...
flatfish : 25/400 proceed...
flatfish : 26/400 proceed...
flatfish : 27/400 proceed...
flatfish : 28/400 proceed...
flatfish : 29/400 proceed...
flatfish : 30/400 proceed...
flatfish : 31/400 proceed...
flatfish : 32/400 proceed...
flatfish : 33/400 proceed...
err in 32
fla

err in 264
flatfish : 266/400 proceed...
flatfish : 267/400 proceed...
flatfish : 268/400 proceed...
flatfish : 269/400 proceed...
flatfish : 270/400 proceed...
flatfish : 271/400 proceed...
flatfish : 272/400 proceed...
err in 271
flatfish : 273/400 proceed...
flatfish : 274/400 proceed...
flatfish : 275/400 proceed...
flatfish : 276/400 proceed...
err in 275
flatfish : 277/400 proceed...
flatfish : 278/400 proceed...
flatfish : 279/400 proceed...
flatfish : 280/400 proceed...
err in 279
flatfish : 281/400 proceed...
flatfish : 282/400 proceed...
flatfish : 283/400 proceed...
err in 282
flatfish : 284/400 proceed...
flatfish : 285/400 proceed...
flatfish : 286/400 proceed...
flatfish : 287/400 proceed...
flatfish : 288/400 proceed...
flatfish : 289/400 proceed...
flatfish : 290/400 proceed...
flatfish : 291/400 proceed...
flatfish : 292/400 proceed...
err in 291
flatfish : 293/400 proceed...
flatfish : 294/400 proceed...
err in 293
flatfish : 295/400 proceed...
err in 294
flatfish : 2

HTTPError: HTTP Error 403: Forbidden